# T011 · Querying online API webservices

> 🚧 This talktorial is still under development. Results hereby presented are subject to change.

Authors:
- Jaime Rodríguez-Guerra, Volkamer lab, Charité
- Dominique Sydow, Volkamer lab, Charité
- David Schaller, Volkamer Lab, Charité

## Aim of this talktorial

In this notebook, you will learn how to programmatically use online web-services from Python, always in the context of drug design. By the end of this talktorial, you will be familiar with REST services and web scraping.

### Contents in Theory

### Contents in Practical

### References

## Theory

### Introduction

Internet is a collection of connected computers that exchange data. In a way, you essentially query machines (servers) with certain parameteres to retrieve specific data. That data will be either:

- **A**. Served straight away, since the server is simply a repository of files (e.g. you [download the ChEMBL database dump](ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/) from their servers).
- **B**. Retrieved from a database and formatted in a particular way. The result you see on your browser is either:
    - **B1**. Pre-processed on the server (e.g. the HTML page you see when you visit any [article in Wikipedia](https://en.wikipedia.org/wiki/Drug_design)).
    - **B2**. Dynamically generated on the client (your browser) as you use the website (e.g. Twitter, Facebook, or any modern _web-app_).
- **C**. Computed through the execution of one or more programs on the server-side (e.g. estimating the protonation states of a protein-ligand complex via [Protoss](https://proteins.plus/help/tutorial)). 

> In a way, configuration C is a special type of B1. You are just replacing the type of task that runs on the server: database querying and HTML rendering _vs_ arbitrary computations that process your query and return data formatted in a domain-specific way.

Another way of categorizing online services is by the _format_ of data returned. Most pages you see on your browser are using HTML, usually focusing on presenting data so it is human-readable. However, some servers might structure that data in a way that is _machine_-readable: this data can be processed in a reliable way because it's formatted using a consistent set of rules that can be easily encoded in a program. Such programs are usually called _parsers_. HTML can be labeled in such a way that data can be obtained reliably, but it is not designed with that purpose in mind. As a result, we will usually prefer using services that provide machine-readable formats, like JSON, CSV or XML.

In practice, both ways of data presentation (should) coexist in harmony. Modern web architecture strives to separate data retrieval tasks from end-user presentation. One popular implementation consists of using a programmatic endpoint that returns machine-readable JSON data, which is then consumed by the user-facing web application (which renders HTML, either on the server -option B1-, or on the user's browser -option B2). Unfortunately, unlike the user-facing applicatino, the programmatic endpoint (API) is not guaranteed to be publicly available, and is sometimes restricted to internal usage in the server side.

In the following sections, we will discuss how to make the most out of each type of online service using Python and some libraries!

> WIP Mark. Pending tasks:
> - Reorder sections below to match types A, B, C above.

#### Online APIs: Application Programming Interfaces

Modern web services are able to provide standardized ways to access their data, especially when it comes to databases. This usually means that you can access a specific URL in their site to request results that are __machine readable__.

For example, [UniProt](https://www.uniprot.org/) is a database for all types of information concerning proteins. If you look for a specific protein, like `Src` (involved in cancer), you will arrive at [this beautiful webpage](https://www.uniprot.org/uniprot/P12931) with well organized content sections. If you add `.fasta` to the URL, however, you will obtain the protein sequence in `FASTA` format.

```
https://www.uniprot.org/uniprot/P12931 -> https://www.uniprot.org/uniprot/P12931.fasta
---

>sp|P12931|SRC_HUMAN Proto-oncogene tyrosine-protein kinase Src OS=Homo sapiens OX=9606 GN=SRC PE=1 SV=3
MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAE
PKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGD
WWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRES
ETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGL
CHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTL
KPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKY
LRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT
ARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVER
GYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL

```

This is a way to provide programmatic access to a web service: through specific URL schemes. However, each web service would have to come up with their own scheme, which will force the developers to implement their scripts on a case-by-case basis.

Fortunately, there are some standardized ways to provide this kind of programmatic access:

- HTTP-based RESTful APIs ([wiki](https://en.wikipedia.org/wiki/Representational_state_transfer#Applied_to_Web_services))
- GraphQL
- SOAP
- gRPC

In this talktorial, we will only use REST and SOAP APIs.

##### HTTP-based RESTful APIs

This type of programmatic access/provision defines a specific entry point for clients (scripts, libraries, programs) that require programmatic access, something like `api.webservice.com`. They can be versioned, so the provider can update the scheme without disrupting existing implementations (`api.webservice.com/v1` will still work even when `api.webservice.com/v2` has been deployed).

This kind of APIs is usually accompanied by well-written documentation explaining all the available actions in the platform. For example, look at the [GitHub REST API for listing repositories](https://developer.github.com/v3/repos/#list-organization-repositories). You can see how every argument and option is documented, along with usage examples. 

The only difficulty is to build the needed URLs in the correct way:

```
https://api.github.com/users/volkamerlab/repos
```

<details>
    <summary>
        Returns (click here!)
    </summary>

```
[
  {
    "id": 156864288,
    "node_id": "MDEwOlJlcG9zaXRvcnkxNTY4NjQyODg=",
    "name": "TeachOpenCADD",
    "full_name": "volkamerlab/TeachOpenCADD",
    "private": false,
    "owner": {
      "login": "volkamerlab",
      "id": 44878588,
      "node_id": "MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4",
      "avatar_url": "https://avatars2.githubusercontent.com/u/44878588?v=4",
      "gravatar_id": "",
      "url": "https://api.github.com/users/volkamerlab",
      "html_url": "https://github.com/volkamerlab",
      "followers_url": "https://api.github.com/users/volkamerlab/followers",
      "following_url": "https://api.github.com/users/volkamerlab/following{/other_user}",
      "gists_url": "https://api.github.com/users/volkamerlab/gists{/gist_id}",
      "starred_url": "https://api.github.com/users/volkamerlab/starred{/owner}{/repo}",
      "subscriptions_url": "https://api.github.com/users/volkamerlab/subscriptions",
      "organizations_url": "https://api.github.com/users/volkamerlab/orgs",
      "repos_url": "https://api.github.com/users/volkamerlab/repos",
      "events_url": "https://api.github.com/users/volkamerlab/events{/privacy}",
      "received_events_url": "https://api.github.com/users/volkamerlab/received_events",
      "type": "Organization",
      "site_admin": false
    },
    "html_url": "https://github.com/volkamerlab/TeachOpenCADD",
    "description": "TeachOpenCADD: a teaching platform for computer-aided drug design (CADD) using open source packages and data",
    "fork": false,
    "url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD",
    "forks_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/forks",
    "keys_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/keys{/key_id}",
    "collaborators_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/collaborators{/collaborator}",
    "teams_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/teams",
    "hooks_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/hooks",
    "issue_events_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues/events{/number}",
    "events_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/events",
    "assignees_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/assignees{/user}",
    "branches_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/branches{/branch}",
    "tags_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/tags",
    "blobs_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/blobs{/sha}",
    "git_tags_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/tags{/sha}",
    "git_refs_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/refs{/sha}",
    "trees_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/trees{/sha}",
    "statuses_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/statuses/{sha}",
    "languages_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/languages",
    "stargazers_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/stargazers",
    "contributors_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/contributors",
    "subscribers_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/subscribers",
    "subscription_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/subscription",
    "commits_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/commits{/sha}",
    "git_commits_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/commits{/sha}",
    "comments_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/comments{/number}",
    "issue_comment_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues/comments{/number}",
    "contents_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/contents/{+path}",
    "compare_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/compare/{base}...{head}",
    "merges_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/merges",
    "archive_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/{archive_format}{/ref}",
    "downloads_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/downloads",
    "issues_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues{/number}",
    "pulls_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/pulls{/number}",
    "milestones_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/milestones{/number}",
    "notifications_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/notifications{?since,all,participating}",
    "labels_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/labels{/name}",
    "releases_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/releases{/id}",
    "deployments_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/deployments",
    "created_at": "2018-11-09T13:15:15Z",
    "updated_at": "2019-07-18T02:36:48Z",
    "pushed_at": "2019-05-03T15:02:03Z",
    "git_url": "git://github.com/volkamerlab/TeachOpenCADD.git",
    "ssh_url": "git@github.com:volkamerlab/TeachOpenCADD.git",
    "clone_url": "https://github.com/volkamerlab/TeachOpenCADD.git",
    "svn_url": "https://github.com/volkamerlab/TeachOpenCADD",
    "homepage": null,
    "size": 28121,
    "stargazers_count": 39,
    "watchers_count": 39,
    "language": "Jupyter Notebook",
    "has_issues": true,
    "has_projects": true,
    "has_downloads": true,
    "has_wiki": true,
    "has_pages": false,
    "forks_count": 13,
    "mirror_url": null,
    "archived": false,
    "disabled": false,
    "open_issues_count": 0,
    "license": {
      "key": "cc-by-4.0",
      "name": "Creative Commons Attribution 4.0 International",
      "spdx_id": "CC-BY-4.0",
      "url": "https://api.github.com/licenses/cc-by-4.0",
      "node_id": "MDc6TGljZW5zZTI1"
    },
    "forks": 13,
    "open_issues": 0,
    "watchers": 39,
    "default_branch": "master"
  }
]
```
</details>

This happens to be a [JSON](https://en.wikipedia.org/wiki/JSON)-formatted dictionary! This is easily parsed into a Python dictionary using the `json` library. The best news is that you don't even need that. Using `requests`, the following operation can be done in three lines:

In [1]:
import requests

response = requests.get("https://api.github.com/users/volkamerlab/repos")
response.raise_for_status()  # this line checks for potential errors
result = response.json()
result

[{'id': 222449034,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyMjI0NDkwMzQ=',
  'name': 'ai_in_medicine',
  'full_name': 'volkamerlab/ai_in_medicine',
  'private': False,
  'owner': {'login': 'volkamerlab',
   'id': 44878588,
   'node_id': 'MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/44878588?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/volkamerlab',
   'html_url': 'https://github.com/volkamerlab',
   'followers_url': 'https://api.github.com/users/volkamerlab/followers',
   'following_url': 'https://api.github.com/users/volkamerlab/following{/other_user}',
   'gists_url': 'https://api.github.com/users/volkamerlab/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/volkamerlab/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/volkamerlab/subscriptions',
   'organizations_url': 'https://api.github.com/users/volkamerlab/orgs',
   'repos_url': 'https://api.github.com/users/volkamerlab/

If you parameterize the URL with an `f-string`, then you can design a function that will list the repositories of any user:

In [2]:
def repos(username):
    response = requests.get(f"https://api.github.com/users/{username}/repos")
    response.raise_for_status()
    return response.json()


repos("volkamerlab")

[{'id': 222449034,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyMjI0NDkwMzQ=',
  'name': 'ai_in_medicine',
  'full_name': 'volkamerlab/ai_in_medicine',
  'private': False,
  'owner': {'login': 'volkamerlab',
   'id': 44878588,
   'node_id': 'MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/44878588?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/volkamerlab',
   'html_url': 'https://github.com/volkamerlab',
   'followers_url': 'https://api.github.com/users/volkamerlab/followers',
   'following_url': 'https://api.github.com/users/volkamerlab/following{/other_user}',
   'gists_url': 'https://api.github.com/users/volkamerlab/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/volkamerlab/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/volkamerlab/subscriptions',
   'organizations_url': 'https://api.github.com/users/volkamerlab/orgs',
   'repos_url': 'https://api.github.com/users/volkamerlab/

##### Generating a client for any API

Did you find that convenient? Well, we are not done yet!

The REST API schema can be expressed programmatically in a document called [Swagger/OpenAPI definitions](https://swagger.io/docs/specification/about/), which allows to dynamically generate a Python client for any REST API that implements the Swagger/OpenAPI schema. [This is the one for GitHub](https://api.apis.guru/v2/specs/github.com/v3/swagger.json).

Of course, there are libraries for doing that in Python:

- `bravado`
- `agithub`

Using Bravado:

In [9]:
from bravado.client import SwaggerClient

# unofficial, lightweight API
GITHUB_SWAGGER = "https://raw.githubusercontent.com/APIs-guru/openapi-directory/eb36b28598eff2cb76008475168e9d0657a51dc4/APIs/github.com/v3/swagger.yaml"
client = SwaggerClient.from_url(GITHUB_SWAGGER)
client

SwaggerClient(https://api.github.com/)

Then, you can have fun inspecting the `client` object for all the API actions as methods. We will see an actual example with [KLIFS](http://klifs.vu-compmedchem.nl) in the case study.

__Tip__: Use `client?` to inspect the client in this Notebook.

Other standards like SOAP operate on a very similar principle: all you need to provide is a definition file to be processed by a client-generating library. For SOAP, the definition files are formatted as `.wsdl`. One of the most popular libraries is `suds` (now installable as `suds-community`).

In [12]:
import suds.client

wsdl = "https://gist.githubusercontent.com/akula1001/9744919/raw/917915efc875effaf0ddb128fe9c18f406f12186/math.wsdl"
client = suds.client.Client(wsdl)
# Suds does not populate the __docstring__ (accessed via ?), but overloads the __str__() method
# In other words, you have to "print" the object to get the documentation
print(client)


Suds ( https://fedorahosted.org/suds/ )  version: 0.8.4

Service ( MathService ) tns="http://example.org/math"
   Prefixes (0)
   Ports (1):
      (MathEndpoint)
         Methods (1):
            sum(xs:double x, xs:double y)
         Types (0):




#### Document parsing

Sometimes you won't be as lucky and the web service won't provide a standardized API that produces machine-readable documents. Instead, you will have to use the regular webpage and parse through the HTML code to obtain the information you need. This is called (web) __scraping__, which usually involves finding the right HTML tags and IDs that contain the valuable data (ignoring things such as the sidebars, top menus, footers, ads, etc).

In scraping, you basically do two things:

1. Access the webpage with `requests` and obtain the HTML contents.
2. Parse the HTML string with `BeautifulSoup` or `requests-html`.


Let's parse the proteinogenic amino acids table in this [Wikipedia article](https://en.wikipedia.org/wiki/Proteinogenic_amino_acid):

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get("https://en.wikipedia.org/wiki/Proteinogenic_amino_acid")
r.raise_for_status()

# To guess the correct steps here, you will have to inspect the HTML code by hand
# Tip: use right-click + inspect content in any webpage to land in the HTML definition ;)
html = BeautifulSoup(r.text)
header = html.find("span", id="General_chemical_properties")
table = header.find_all_next()[4]
table_body = table.find("tbody")

data = []
for row in table_body.find_all("tr"):
    cells = row.find_all("td")
    if cells:
        data.append([])
    for cell in cells:
        cell_content = cell.text.strip()
        try:  # convert to float if possible
            cell_content = float(cell_content)
        except ValueError:
            pass
        data[-1].append(cell_content)
pd.DataFrame.from_records(data)

,0,1,2,3,4,5
0,A,Ala,89.09404,6.01,2.35,9.87
1,C,Cys,121.15404,5.05,1.92,10.7
2,D,Asp,133.10384,2.85,1.99,9.9
3,E,Glu,147.13074,3.15,2.1,9.47
4,F,Phe,165.19184,5.49,2.2,9.31
5,G,Gly,75.06714,6.06,2.35,9.78
6,H,His,155.15634,7.6,1.8,9.33
7,I,Ile,131.17464,6.05,2.32,9.76
8,K,Lys,146.18934,9.6,2.16,9.06
9,L,Leu,131.17464,6.01,2.33,9.74


#### Browser remote control

The trend some years ago was to build servers that dynamically-generated HTML documents with some JavaScript here and there (such as Wikipedia). In other words, the HTML is built in the server and sent to the client (your browser).

However, latest trends are pointing towards full applications built entirely with JavaScript frameworks. This means that the HTML content is dynamically generated in the client. Traditional parsing won't work and you will only download the placeholder HTML code that hosts the JavaScript framework. To work around this, the HTML must be rendered with a client-side JavaScript engine.

We won't cover this in the current notebook, but you can check the following projects if you are interested:

- [puppeteer](https://github.com/GoogleChrome/puppeteer)
- [selenium](https://www.seleniumhq.org/)

## Practical

***

## Discussion

In this theoretical introduction you have seen how different methods to programmatically access online web services can be used from a Python interpreter. Leveraging these techniques you will be able to build automated pipelines inside Jupyter Notebooks. Keep reading on the following parts of this talktorial to see an example applied to CADD.

## Quiz

* How can you find the correct HTML tags and identifiers to scrape a specific part of a website? Can it be automated?
* Would you rather use programmatic APIs or manually crafted scrapers?